In [1]:
import geopandas as gpd
import line_orchard_detection as ld

In [2]:
path = '../datasets/area_estudo_teste1/'

pontos = gpd.read_file(path + 'centroids.geojson')

linhas, pts = ld.detect_lines_uni(pontos, row_inverse=False, pt_inverse=False)

linhas.to_file(path + 'line_prediction.geojson', driver='GeoJSON')
pts.to_file(path + 'centroides_area_mapped.geojson', driver='GeoJSON')
# falhas.to_file(path + 'falhas_prediction.geojson', driver='GeoJSON')

ValueError: Invalid call for scalar access (setting)!